In [1]:
import numpy as np
import pandas as pd
import random as rnd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
import re 
import time

In [2]:
#base_challengers_df = pd.read_csv('../LOL_dataset/challenger_match_V2.csv')

In [3]:
base_match_df = pd.read_pickle('../LOL_dataset/match_data_version1.pickle')

### Cleaning up the match dataframe dropping columns and removing rows that don't apply

In [28]:
base_match_df['gameId'] = base_match_df['gameId'].astype('int64')

In [29]:
base_match_df.drop_duplicates('gameId',inplace=True)

In [30]:
trim_match_df = base_match_df.drop(columns = ['gameCreation', 'gameType', 'gameType', 'gameVersion',
                                              'participantIdentities', 'platformId', 'queueId', 'status.message',
                                              'status.status_code' ])

In [31]:
trim_match_df['gameId'] = trim_match_df['gameId'].astype('int64')

In [32]:
trim_match_df = trim_match_df[(trim_match_df['gameMode']=='CLASSIC') & (trim_match_df['mapId']==11.0) & (trim_match_df['seasonId'] == 13.0)]

In [33]:
trim_match_df.drop(columns= ['gameMode', 'mapId', 'seasonId'], inplace = True )

In [34]:
trim_match_df = trim_match_df[['gameId', 'participants', 'gameDuration']]

In [35]:
trim_match_df.shape

(92649, 3)

### Cleaning up winning and losing dataframes to be able to reduce them

In [36]:
base_winner_df =  pd.read_pickle('../LOL_dataset/match_winner_data_version1.pickle')

In [37]:
base_winner_df.drop_duplicates('gameId',inplace=True)

In [38]:
base_loser_df = pd.read_pickle('../LOL_dataset/match_loser_data_version1.pickle')

In [39]:
base_loser_df.drop_duplicates('gameId',inplace=True)

In [40]:
trim_winner_df = base_winner_df
trim_loser_df = base_loser_df

In [41]:
trim_winner_df.drop(columns= ['vilemawKills', 'dominionVictoryScore', 'bans'], inplace=True)
trim_loser_df.drop(columns= ['vilemawKills', 'dominionVictoryScore', 'bans'], inplace=True)

In [42]:
trim_winner_df['gameId'] = trim_winner_df['gameId'].astype('int64')
trim_loser_df['gameId'] = trim_loser_df['gameId'].astype('int64')

In [43]:
trim_winner_df = pd.DataFrame(trim_winner_df[trim_winner_df['gameId'].isin(trim_match_df['gameId'])])
trim_loser_df = pd.DataFrame(trim_loser_df[trim_loser_df['gameId'].isin(trim_match_df['gameId'])])

In [44]:
df = pd.DataFrame(index= trim_match_df['gameId'])
df.sort_index(inplace=True)

In [45]:
trim_winner_df.set_index('gameId',inplace=True)
trim_loser_df.set_index('gameId', inplace=True)

In [46]:
trim_winner_df.sort_index(inplace=True)
trim_loser_df.sort_index(inplace=True)

In [47]:
trim_winner_df.shape

(92649, 13)

### Contructing the final dataframe

In [48]:
df['Winner'] = trim_winner_df['teamId']

In [49]:
trim_match_df.set_index('gameId', inplace=True)
trim_match_df.sort_index(inplace=True)
df["Match Length"] = round((trim_match_df['gameDuration']/60), ndigits=1)

In [50]:
df[['Winner First Blood', 'Winner First Tower', 'Winner First Inhibitor', 'Winner First Baron', 'Winner First Dragon',
   'Winner First Herald']] = trim_winner_df[['firstBlood', 'firstTower', 'firstInhibitor', 'firstBaron', 'firstDragon', 'firstRiftHerald']]

In [52]:
blue_winner = pd.DataFrame()
blue_loser = pd.DataFrame
blue_df = pd.DataFrame()
red_winner = pd.DataFrame()
red_loser = pd.DataFrame
red_df = pd.DataFrame()

In [53]:
blue_winner = trim_winner_df[trim_winner_df['teamId'] == 100]
blue_loser = trim_loser_df[trim_loser_df['teamId'] == 100]
red_winner = trim_winner_df[trim_winner_df['teamId'] == 200]
red_loser = trim_loser_df[trim_loser_df['teamId'] == 200]

In [54]:
blue_winner = pd.DataFrame(blue_winner.drop(columns = ['firstBlood', 'firstTower','firstInhibitor',
                                                       'firstBaron', 'firstDragon', 'firstRiftHerald']))
red_winner = pd.DataFrame(red_winner.drop(columns = ['firstBlood', 'firstTower','firstInhibitor',
                                                       'firstBaron', 'firstDragon', 'firstRiftHerald']))

In [55]:
blue_loser = pd.DataFrame(blue_loser.drop(columns = ['firstBlood', 'firstTower','firstInhibitor',
                                                       'firstBaron', 'firstDragon', 'firstRiftHerald']))
red_loser = pd.DataFrame(red_loser.drop(columns = ['firstBlood', 'firstTower','firstInhibitor',
                                                       'firstBaron', 'firstDragon', 'firstRiftHerald']))

In [56]:
blue_df = pd.concat([blue_winner, blue_loser])
red_df = pd.concat([red_winner, red_loser])

In [57]:
blue_df.sort_index(inplace=True)
red_df.sort_index(inplace=True)

In [58]:
df[['Winner First Blood', 'Winner First Tower', 'Winner First Inhibitor', 'Winner First Baron', 'Winner First Dragon',
   'Winner First Herald']] = trim_winner_df[['firstBlood', 'firstTower', 'firstInhibitor', 'firstBaron', 'firstDragon', 'firstRiftHerald']]

In [59]:
blue_df.rename(columns= {'towerKills': 'Blue Tower Kills', 'inhibitorKills': 'Blue Inhibitor Kills', 'baronKills': ' Blue Baron Kills',
                        'dragonKills' : 'Blue Dragon Kills', 'riftHeraldKills' : 'Blue Rift Herald Kills'}, inplace=True)
red_df.rename(columns= {'towerKills': 'Red Tower Kills', 'inhibitorKills': 'Red Inhibitor Kills', 'baronKills': ' Red Baron Kills',
                        'dragonKills' : 'Red Dragon Kills', 'riftHeraldKills' : 'Red Rift Herald Kills'}, inplace=True)

In [60]:
blue_df.drop(columns =['win', 'teamId'],inplace=True)
red_df.drop(columns =['win', 'teamId'],inplace=True)

In [61]:
both_counts_df = pd.merge(blue_df, red_df, how = 'outer', on = 'gameId')

In [62]:
both_counts_df.shape

(92649, 10)

In [63]:
df *= 1

In [65]:
df.head(3)

,Winner,Match Length,Winner First Blood,Winner First Tower,Winner First Inhibitor,Winner First Baron,Winner First Dragon,Winner First Herald
gameId,,,,,,,,
3508539941,200,35.3,0,0,1,1,1,0
3510680882,200,30.9,1,0,1,1,0,0
3510731607,100,29.0,1,1,1,1,1,1


In [66]:
df = pd.merge(df, both_counts_df, on= 'gameId')

In [71]:
df['Winner'] = df['Winner'].map({100:'Blue', 200:'Red'})

In [72]:
df.head(3)

,Winner,Match Length,Winner First Blood,Winner First Tower,Winner First Inhibitor,Winner First Baron,Winner First Dragon,Winner First Herald,Blue Tower Kills,Blue Inhibitor Kills,Blue Baron Kills,Blue Dragon Kills,Blue Rift Herald Kills,Red Tower Kills,Red Inhibitor Kills,Red Baron Kills,Red Dragon Kills,Red Rift Herald Kills
gameId,,,,,,,,,,,,,,,,,,
3508539941,Red,35.3,0,0,1,1,1,0,4,0,0,2,1,6,1,1,2,0
3510680882,Red,30.9,1,0,1,1,0,0,7,2,0,2,1,5,2,1,1,0
3510731607,Blue,29.0,1,1,1,1,1,1,10,2,1,2,1,1,0,0,1,0
